In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from bokeh.io import output_file, output_notebook, show, export_png
from bokeh.models import ColumnDataSource
from bokeh.palettes import GnBu3, OrRd3
from bokeh.plotting import figure
from scipy.stats import wilcoxon

from dnafiber import get_model
from dnafiber.data.intergrader import BLIND_MAPPING, REVERSE_BLIND_MAPPING

from dnafiber.data.intergrader.io import read_mask_from_path_gens
from dnafiber.data.intergrader.plot import imshow_compare


2025-08-20 16:58:05.869 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-20 16:58:06.071 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-20 16:58:06.072 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


ImportError: cannot import name 'get_model' from 'dnafiber' (/home/clement/Documents/Projets/DNAFiber/dnafiber/__init__.py)

In [8]:
df = pd.read_pickle("properties.pkl")


In [5]:
df = df[df.fiber_type != "double"]

In [9]:
df

,annotator,image_type,red,green,ratio,fiber_type,S,H,Y,C,common
0,S,siMMS22L_+dox-01,86,129,6.666667e-01,double,True,True,False,False,2
1,S,siMMS22L_+dox-01,226,211,1.071090e+00,multiple,True,False,False,False,1
2,S,siMMS22L_+dox-01,110,125,8.800000e-01,double,True,True,True,True,4
3,S,siMMS22L_+dox-01,21,38,5.526316e-01,double,True,False,True,True,3
4,S,siMMS22L_+dox-01,17,0,1.700000e+11,single,True,True,True,True,4
...,...,...,...,...,...,...,...,...,...,...,...
457,C,siBRCA2-01,20,27,7.407407e-01,multiple,False,True,False,True,2
458,C,59_siBRCA2-02,10,35,2.857143e-01,double,False,False,False,True,1
459,C,59_siBRCA2-02,17,37,4.594595e-01,double,True,False,False,True,2
460,C,59_siBRCA2-02,32,41,7.804878e-01,double,False,False,False,True,1


In [10]:
# Counts the number of occurences of each annotators
results = dict()
for annotator in df["annotator"].unique():
    df_annotator = df.loc[df["annotator"] == annotator]
    value_count = df_annotator["common"].value_counts()
    results[annotator] = (list(value_count.sort_index()), len(df_annotator))


output_notebook()
if len(df["annotator"].unique()) == 5:
    colors = ["#d20f39"]
else:
    colors = []
colors += ["#e78284", "#f5a97f", "#eed49f", "#a6d189"]
annotators = list(results.keys())
values = [results[annotator][0] for annotator in annotators]
data = {f"Found by {i + 1}": [int(v[i]) for v in values] for i in range(len(values[0]))}
y = list(data.keys())
data["annotators"] = annotators

p = figure(
    title="Counts of common fibers by annotator",
    x_range=annotators,
    height=400,
    tooltips="$name @annotators: @$name",
    toolbar_location=None,
)

p.vbar_stack(
    y,
    x="annotators",
    width=0.9,
    source=data,
    color=colors,
    legend_label=y,
)
for i, a in enumerate(annotators):
    total = 0
    for v in values[i]:
        p.text(
            x=[a],
            y=v + total - 5,
            text=[str(v)],
            text_color="#24273a",
            text_align="center",
            text_baseline="middle",
            angle=0,
        )
        total += v

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

export_png(p, filename="common_fibers_only_double.png", width=800, height=400)

Loading BokehJS ...

'/home/clement/Documents/Projets/DNAFiber/notebook/intergraders/common_fibers_only_double.png'

In [ ]:
data["Found by 1"]

[8, 13, 4, 2]